# Free Comments Classifcation

This script is designed to carry out a Text analysis of free comments to classify them into respective categories that they allude to. This will help draw insights of what elements are of concern to customers by how much they are highlighted in the customers' comments.

***
## Background

## Advisory K
**Advisory K** is a wholly owned Zimbabwean firm, and the first and only official partner in Zimbabwe of [Mercer Global](www.mercer.com) – the largest and leading HR Consulting firm in the world. Through our enriched experience, and our access to global best practice and solutions in Strategy, Organisation and People solutions, we are the preeminent firm in the country delivering cohesive strategy and organisational services focused on the public and parastatal sectors.

### We help our cients know more and do more
Through our five practice areas we help our clients **Know More (Research & Analytics)** so that they can **Do More (Strategy, Organisation and People)**. We have a strong belief in the value of data and in data-driven decision making in an organisation. Our suite of services is designed to ensure that reliable data drives each part of any organisation as we take it from Knowing to Doing.  
![image.png](PIcture1.png)

### Research and Analytics
The **Research and Analytics** Practice Areas provides our clients with data driven strategic insights. Our data driven approach ensures **subjectivism is kept to a minimum** and **FACTS** are a major component of our results.

As the **Research and Analytics** arm of **Advisory K**, one of the several products we offer is **Survey Research** specialising in:
1. Survey Design and Deployment
2. Survey Administering
3. Survey Analysis and Reporting

## Survey Analysis
Making sense of responses. Tie-up all different responses to come up with an objective actionable insight of what the respondents perceive as a group. For all questions in a survey where responses are measured by a numerical scale this is easy as an index for the perception can be calculated, for instance a Customer Satisfcation Index (CSI).
However, for free comments, it is difficult to make sense of a large amount of various responses addressing various concerns.

### Problem Case 
How do we efficiently identify the concerns and suggestions that customers raise in the free comment section of the survey?

### Project Objective
Create a model for Text analysis of Customer Satisfaction Survey free comments and suggestions.

### Project approach
1.	Research on text analysis and the various algorithms used. 
2.	Evaluate the different algorithms and choose one that best addresses the need taking into consideration:
    * quantity of data
    * scalability
    * continuity
3.	Implement chosen algorithm from custom template
4.	Create an executable script that is easy to use


****
## Set up... 

### Tools used.
I created the model using Jupyter Notebook. This a powerful tool that combines:
* program code 
* program code output 
* visualisations
* narrative text 
* mathematical equations
* and media eg pictures.

I chose to do the project in Python since it is the programming language of choice in Data science. The language also has an extensive range of ready to use libraries and modules that greatly shorten time to program and allow you to get results faster.

**Step 1:** Import the relevant modules and libraries.

In [1]:
#Importing libraries

import logging 
import gensim 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk

from gensim import corpora 
from gensim import models 
from gensim import similarities 
from nltk.corpus import stopwords 
from nltk.tokenize import wordpunct_tokenize 
from collections import defaultdict 
from sklearn.cluster import KMeans 
from time import time 
from pprint import pprint

#Settings

sns.set(context='notebook', style='whitegrid', palette='deep', rc=None) 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Data set
I will be analysing the comments that were drawn from monthly Customer Satiscation Surveys from January 2016 to January 2018, for OneClient who is a healthcare service provider (name changed to not reveal Advisory K client information). 
One challenge encountered was getting enough data since most respondents do not take time to complete the free comments section of the survey sinces it is optional and respondents are not obliged to respond. We need to cultivate a culture of giving feedback without fearing being too critical and judgemental. 

**Step 2:** Loading the data

In [2]:
# Loading the data from a file
comments = pd.read_csv('all_comments_2016-2018-Copy1.csv')

***
## Pre-processing

### Text processing

To begine our analysis we first have to change the comments into a [**Text Corpus**](http://language.worldofcomputing.net/linguistics/introduction/what-is-corpus.html). This is a large and structured set of machine readable text that is representative of or some language, stored and processed electronically.

To attain our corpus:
1. First remove irrelevant text. This can be done by utilizing the stopwords function of the nltk module which removes irrelevant words for a given language. Stopwords can be expanded to include words or symbols in text that might be considered as irrelevant by simply updating the function with the words to discard. 
2. Secondly we break up the filtered texts into individual words. This is called [Tokenization](https://www.techopedia.com/definition/13698/tokenization).
3. We further filter the list of tokens nested within the list of text to remove the words with low frequency. These words rarely belong to the context and do not offer much insight.
4. Create a dictionary that contains the cleaned data (in form of a list) ready to convert into a corpus (bag-of-words). Each word in the corpus is represented by a unique word id and respective weight.

In [3]:
# Text corpus

#put file comments into a list.
document = [i for i in comments.comments]
#pprint(document)

#Removing common words and tokenizing
stop_words = set(stopwords.words('english'))
for doc in document:
    list_of_words = [i.lower() for i in wordpunct_tokenize(doc)
                    if i.lower() not in stop_words]
stop_words.update(list_of_words)

# Removing common words
texts = [[word for word in doc.lower().split()
         if word not in stop_words]
        for doc in document]

# Removing words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] +=1
texts = [[token for token in text
         if frequency[token] > 7]
        for text in texts]

# Create dictionary of the document
dictionary = corpora.Dictionary(texts)
dictionary.save('allcomments.dict')

# Converting document to a vector (bag-of-words)
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('allcomments.mm', corpus)

# Print done
print('Done!')

2018-07-27 08:45:43,234 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-27 08:45:43,281 : INFO : built Dictionary(415 unique tokens: ['services', 'polite', 'staff', 'customers', 'space']...) from 3251 documents (total 12267 corpus positions)
2018-07-27 08:45:43,283 : INFO : saving Dictionary object under allcomments.dict, separately None
2018-07-27 08:45:43,306 : INFO : saved allcomments.dict
2018-07-27 08:45:43,353 : INFO : storing corpus in Matrix Market format to allcomments.mm
2018-07-27 08:45:43,355 : INFO : saving sparse matrix to allcomments.mm
2018-07-27 08:45:43,359 : INFO : PROGRESS: saving document #0
2018-07-27 08:45:43,387 : INFO : PROGRESS: saving document #1000
2018-07-27 08:45:43,412 : INFO : PROGRESS: saving document #2000
2018-07-27 08:45:43,436 : INFO : PROGRESS: saving document #3000
2018-07-27 08:45:43,449 : INFO : saved 3251x415 matrix, density=0.881% (11889/1349165)
2018-07-27 08:45:43,451 : INFO : saving MmCorpus index to allcomments.mm.in

Done!


In [16]:
#texts

In [18]:
#corpus

In [4]:
# Loading
dictionary = corpora.Dictionary.load('allcomments.dict')


2018-07-27 08:45:43,470 : INFO : loading Dictionary object from allcomments.dict
2018-07-27 08:45:43,476 : INFO : loaded allcomments.dict


***
## Transformation: Term Frequency - Inverse Document Frequency (TF-IDF)

**tf-idf** is initialized with a **bag-of-words** (integer values) training corpus and takes in a vector, returning another vector of the same dimensionality. except that features which are rare in the training corpus will have their value increased. It therefore converts integer-valued vectors into real-valued ones, while leaving the number of dimensions intact. It can also, optionally, normalize the resulting vectors to (Euclidean) unit length.

#### *Step 1:*

In [5]:
# Initialization
tfidf = models.TfidfModel(corpus)

2018-07-27 08:45:43,486 : INFO : collecting document frequencies
2018-07-27 08:45:43,491 : INFO : PROGRESS: processing document #0
2018-07-27 08:45:43,508 : INFO : calculating IDF weights for 3251 documents and 414 features (11889 matrix non-zeros)


#### *Step 2:*

In [6]:
# Applying the transformation to the whole corpus
corpus_tfidf = tfidf[corpus]

***
## Transformation: Latent Semantic Indexing (LSI)
[Latent semantic indexing (LSI)](https://en.wikipedia.org/wiki/Latent_semantic_analysis) is an indexing and retrieval method that uses a mathematical technique called [singular value decomposition (SVD)](https://en.wikipedia.org/wiki/Singular-value_decomposition) to identify patterns in the relationships between terms and concepts contained in an unstructured collection of text. A key feature of LSI is its ability to extract the conceptual content of a body of text by establishing associations between those terms that occur in similar contexts.

### Implemented to correlate semantically related terms that are latent in a collection of text.

Transforms documents from either **bag-of-words** or (preferably) **tf-idf-weighted** space into a latent space of a lower dimensionality.

**LSI** is unique in that we can continue "training" at any point, simply by providing more training documents. This is done by incremental updates to the underlying model, in a process called *online training*. Because of this feature, the input document stream may even be infinite - just keep feeding **LSI** new documents as they arrive, while using the computed transformation model as read-only in the meanwhile!

[*gensim.models.lsimodel*](https://radimrehurek.com/gensim/models/lsimodel.html#module-gensim.models.lsimodel) contains details for making an **LSI** model gradually "forget" old observations in infinite streams, with parameters that can be tweaked to affect speed, memory footprint and numerical precision of the **LSI** algorithm.

[*gensim*](https://radimrehurek.com/gensim) uses a novel online incremental streamed distributed training algorithm published [here](https://radimrehurek.com/gensim/tut2.html#id10). *gensim* also executes a stochastic multi-pass algorithm from [Halko et al.](https://radimrehurek.com/gensim/tut2.html#id9) internally, to accelerate in-core part of the computations.

In [7]:
# Initializing an LSI transformation
lsi = models.LsiModel(corpus_tfidf, id2word = dictionary, num_topics = 10)
corpus_lsi = lsi[corpus_tfidf]

2018-07-27 08:45:43,530 : INFO : using serial LSI version on this node
2018-07-27 08:45:43,533 : INFO : updating model with new documents
2018-07-27 08:45:43,726 : INFO : preparing a new chunk of documents
2018-07-27 08:45:43,764 : INFO : using 100 extra samples and 2 power iterations
2018-07-27 08:45:43,766 : INFO : 1st phase: constructing (415, 110) action matrix
2018-07-27 08:45:43,991 : INFO : orthonormalizing (415, 110) action matrix
2018-07-27 08:45:44,526 : INFO : 2nd phase: running dense svd on (110, 3251) matrix
2018-07-27 08:45:44,797 : INFO : computing the final decomposition
2018-07-27 08:45:44,812 : INFO : keeping 10 factors (discarding 64.603% of energy spectrum)
2018-07-27 08:45:44,840 : INFO : processed documents up to #3251
2018-07-27 08:45:44,860 : INFO : topic #0(14.283): -1.000*"#name?" + 0.000*"patients." + 0.000*"bit" + 0.000*"us" + 0.000*"machines" + -0.000*"wait" + -0.000*"doctors." + -0.000*"rude" + 0.000*"availability" + 0.000*"nurses"
2018-07-27 08:45:44,862 

In [8]:
lsi.print_topics(5)

2018-07-27 08:45:44,890 : INFO : topic #0(14.283): -1.000*"#name?" + 0.000*"patients." + 0.000*"bit" + 0.000*"us" + 0.000*"machines" + -0.000*"wait" + -0.000*"doctors." + -0.000*"rude" + 0.000*"availability" + 0.000*"nurses"
2018-07-27 08:45:44,903 : INFO : topic #1(10.397): -0.715*"good." + -0.598*"service" + -0.280*"far" + -0.139*"good" + -0.105*"services" + -0.087*"keep" + -0.057*"slow" + -0.039*"improve" + -0.039*"up." + -0.037*"medication"
2018-07-27 08:45:44,908 : INFO : topic #2(8.096): -0.358*"medication" + -0.349*"frames" + -0.235*"keep" + -0.230*"time" + -0.229*"waiting" + 0.228*"good." + -0.224*"good" + -0.211*"improve" + -0.205*"need" + 0.203*"far"
2018-07-27 08:45:44,918 : INFO : topic #3(7.729): -0.518*"keep" + -0.451*"good" + -0.333*"service" + 0.307*"far" + 0.265*"good." + 0.257*"frames" + 0.182*"medication" + 0.124*"waiting" + 0.120*"time" + 0.113*"improve"
2018-07-27 08:45:44,923 : INFO : topic #4(7.335): -0.734*"medication" + 0.488*"frames" + -0.213*"shortages" + 0.1

[(0,
  '-1.000*"#name?" + 0.000*"patients." + 0.000*"bit" + 0.000*"us" + 0.000*"machines" + -0.000*"wait" + -0.000*"doctors." + -0.000*"rude" + 0.000*"availability" + 0.000*"nurses"'),
 (1,
  '-0.715*"good." + -0.598*"service" + -0.280*"far" + -0.139*"good" + -0.105*"services" + -0.087*"keep" + -0.057*"slow" + -0.039*"improve" + -0.039*"up." + -0.037*"medication"'),
 (2,
  '-0.358*"medication" + -0.349*"frames" + -0.235*"keep" + -0.230*"time" + -0.229*"waiting" + 0.228*"good." + -0.224*"good" + -0.211*"improve" + -0.205*"need" + 0.203*"far"'),
 (3,
  '-0.518*"keep" + -0.451*"good" + -0.333*"service" + 0.307*"far" + 0.265*"good." + 0.257*"frames" + 0.182*"medication" + 0.124*"waiting" + 0.120*"time" + 0.113*"improve"'),
 (4,
  '-0.734*"medication" + 0.488*"frames" + -0.213*"shortages" + 0.144*"long" + -0.129*"shortage" + -0.116*"pharmacy" + 0.114*"waiting" + 0.112*"time" + 0.111*"keep" + 0.108*"spectacles"')]

### Ouput 1:

| Top 5 Topics from Latent Semantic Indexing (LSI) |
| --- |
| **1** | *"#name", "enough", "come", "2", "sometimes", "took", "needed", "take", "receptionist", "ambulance"* | **Description:** | Comments related to |
| **2** | *"good", "service", "far", "good", "services", "keep", "slow", "improve", "up", "medication"* | **Description:** | Comments related to medication with references to "good service but with need to improve speed of delivery". |
| **3** | *"medication", "frames", "keep", "time", "waiting", "good", "good", "improve", "need", "far"* | **Description:** | Comments related to spectatcle frames with reference to "them being good but with need to improve on the waiting time and distance". |
| **4** | *"keep", "good", "service", "far", "good", "frames", "medication", "waiting", "time", "improve"* | **Description:** | Comments with feedback to maintain good service but highlight need to improve |
| **5** | *"medication", "frames", "shortages", "long", "shortage", "pharmacy", "waiting", "time", "keep", "spectacles"* | **Description:** | Comments related to medication and spectacle frames with feedback on shortages in pharmacy and long waiting times. |

In [9]:
# Saving
lsi.save('allcomments.lsi')

2018-07-27 08:45:44,954 : INFO : saving Projection object under allcomments.lsi.projection, separately None
2018-07-27 08:45:44,959 : INFO : saved allcomments.lsi.projection
2018-07-27 08:45:44,960 : INFO : saving LsiModel object under allcomments.lsi, separately None
2018-07-27 08:45:44,963 : INFO : not storing attribute projection
2018-07-27 08:45:44,965 : INFO : not storing attribute dispatcher
2018-07-27 08:45:44,967 : INFO : saved allcomments.lsi


In [10]:
# Loading
lsi = models.LsiModel.load('allcomments.lsi')

2018-07-27 08:45:44,977 : INFO : loading LsiModel object from allcomments.lsi
2018-07-27 08:45:44,980 : INFO : loading id2word recursively from allcomments.lsi.id2word.* with mmap=None
2018-07-27 08:45:44,983 : INFO : setting ignored attribute projection to None
2018-07-27 08:45:44,986 : INFO : setting ignored attribute dispatcher to None
2018-07-27 08:45:44,987 : INFO : loaded allcomments.lsi
2018-07-27 08:45:44,989 : INFO : loading LsiModel object from allcomments.lsi.projection
2018-07-27 08:45:44,991 : INFO : loaded allcomments.lsi.projection


***
## Similarity Queries
Used to determine similarity between pairs of documents, or the similarity between a specific document and a set of other documents (such as a user query vs. indexed documents).

#### *Step1:*

In [11]:
# Initializing the query structure: transform corpus to LSI space
# and index it
index = similarities.MatrixSimilarity(lsi[corpus], num_features = 10)

# Saving
index.save('allcomments.index')

2018-07-27 08:45:45,010 : INFO : creating matrix with 3251 documents and 10 features
/Users/kudakwashe/anaconda3/envs/Virtual1/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-07-27 08:45:45,393 : INFO : saving MatrixSimilarity object under allcomments.index, separately None
2018-07-27 08:45:45,396 : INFO : saved allcomments.index


In [12]:
# Loading
index = similarities.MatrixSimilarity.load('allcomments.index')

2018-07-27 08:45:45,405 : INFO : loading MatrixSimilarity object from allcomments.index
2018-07-27 08:45:45,442 : INFO : loaded allcomments.index


#### *Step2:*

In [13]:
# Performing queries
doc = "pharmacy"
vec_bow = dictionary.doc2bow(doc.lower().split())

# Convert the query to LSI spaace
vec_lsi = lsi[vec_bow]

# Perform a similarity query against the corpus
sims = index[vec_lsi]

# Ranking the comments by their weights of similarity
sims = sorted(enumerate(sims), key=lambda item: -item[1])

# Printing the associated comments:
for i in range(5):
    print("Comments Rank #{}:\tWeights {}\nRaw Text: {}\n".format(i+1,
            sims[i][1],document[sims[i][0]]))

Comments Rank #1:	Weights 0.9999999403953552
Raw Text: pharmacy in town doesn't  have medicine.

Comments Rank #2:	Weights 0.9997751712799072
Raw Text: no pharmacy in chipinge

Comments Rank #3:	Weights 0.9997751712799072
Raw Text: establish a pharmacy in chipinge

Comments Rank #4:	Weights 0.9994443655014038
Raw Text: pharmacy should have medication.

Comments Rank #5:	Weights 0.9981115460395813
Raw Text: they should open a pharmacy in chipinge 



### Output 2:

||Top 5 Comments from Similarity Query of document 'Pharmacy' |
| --- |
| **1** |  pharmacy in town doesn't  have medicine. |
| **2** |  no pharmacy in chipinge |
| **3** |  establish a pharmacy in chipinge |
| **4** |  pharmacy should have medication. |
| **5** |  they should open a pharmacy in chipinge |

### Observations and Results

At this point, we observe that **LSI** provides topic models with a coherent semantic reference. In this instance it is the best model to use compared to another model known as **Latent Dirichlet Allocation (LDA)** because **LSI** is less sensitive to the amount of data than **LDA**, given we are working with just over 3000 comments to conduct our analysis. However, with more comments (apprximately 10000 comments) **LDA** tends to provide more better results than **LSI** 

**LSI** with the ability to query for similarity allows to bring out insights on what the comments highlight about a certain topics.

Given enough computing power and other relevant resources, **Clustering** using the K-means can be done to better categorize the comments with respect to specified categories. These can then be also categorized by sentiment (positive, negative, neutral). 

### Value Proposition
* Increase efficiency of analysing surveys
* Cost reductions to the firm in terms of billable hours to analyse comments
* Reduce subjectivity of an individual's interpretation of comments

### Sustainability
* The resultant script will be repeatedly used for Text analysis of the CSS free comments by the Advisory K research and analytics department. This is going to greatly enhance the firm’s reports on CSS and create added value of their offering. 
* The script is also the basis of conducting Sentiment Analysis. It can be scaled to gauge the perception of clients (positive, negative or neutral) with respect to identified categories



### Challenges and Opportunities
The project was challenging as I was not aware of the concept of text or sentiment analysis. However, it proposed a valuable opportunity to learn something new and required me to do some programming, which for a MIS major I had not managed to do all summer. 
As an aspiring data analyst and business consultant, I got to learn more about the different branches of analytics and get a picture of the opportunities that exist there.
